# Imports

In [ ]:
import tensorflow as tf
from keras.models import load_model
from keras.utils import plot_model
from time import time as t
from tqdm import tqdm
import cv2
from glob import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec
%matplotlib inline

## Helper Functions

In [ ]:
# df = pd.read_csv('veriwild_ann.csv')
# model_lables = df.drop('image',axis=1).columns[:151]
# type_lables = df.drop('image',axis=1).columns[151:-12]
# color_lables = df.drop('image',axis=1).columns[-12:]

def decodeLables(encoding):
    idx = encoding.split('-')
    return model_lables[int(idx[0])]+'-'+type_lables[int(idx[1])]+'-'+color_lables[int(idx[2])]

def decodeOutput(idx):
    lable = decodeLables(idx[3])
    image = cv2.resize(cv2.imread('raw/{}.jpg'.format(idx[0])),(100,100),cv2.INTER_AREA)
    _id = idx[1]
    return lable, image, _id


In [ ]:
class Node():
    def __init__(self,nodeName, model_name='model/VMMR_noOPT.hdf5', csv_files='veriwild_ann.csv'):
        self.nodeName = nodeName
        self.model = load_model(model_name, compile=False)
        df = pd.read_csv(csv_file)
        self.model_lables = df.drop('image',axis=1).columns[:151]
        self.type_lables = df.drop('image',axis=1).columns[151:-12]
        self.color_lables = df.drop('image',axis=1).columns[-12:]
        
    def read(self, image_name):
        return cv2.resize(cv2.imread(image_name),(299,299),cv2.INTER_AREA)
    def predict(self, img):
        pred = self.model.predict(img.reshape(-1,299,299,3)/255.0)
        _id =  '{}-{}-{}'.format(np.argmax(pred[0]),np.argmax(pred[1]),np.argmax(pred[2]))
        text = '{}-{}-{}'.format(self.model_lables[np.argmax(pred[0])],
                                 self.type_lables[np.argmax(pred[1])],
                                 self.color_lables[np.argmax(pred[2])])
        return _id,text
    def process(self):
        self.ref_time = t()
        i = 0
        for name in tqdm(glob('raw/{}/*.jpg'.format(self.name))):
            img = self.read(name.replace('\\','/'))
            pred,text = self.predict(img)
            n = name.split('\\')[1].split('.')[0]
            print(pred, text)

In [ ]:
node1 = Node('node_1')
node1.process()